# İş Problemi

ID'si verilen kullanıcı için item-based ve user-based recommender yöntemlerini kullanarak 10 film önerisi yapınız.

# Veri Seti Hikayesi

Veri seti, bir film tavsiye hizmeti olan MovieLens tarafından sağlanmıştır. İçerisinde filmler ile birlikte bu filmlere yapılan
derecelendirme puanlarını barındırmaktadır. 27.278 filmde 2.000.0263 derecelendirme içermektedir. Bu veri seti ise 17 Ekim 2016
tarihinde oluşturulmuştur. 138.493 kullanıcı ve 09 Ocak 1995 ile 31 Mart 2015 tarihleri arasında verileri içermektedir. Kullanıcılar
rastgele seçilmiştir. Seçilen tüm kullanıcıların en az 20 filme oy verdiği bilgisi mevcuttur.

# User Based Recommendation

## Görev 1: Veriyi Hazırlama

### Adım 1:
movie, rating veri setlerini okutunuz.

In [308]:
import pandas as pd

In [309]:
movie = pd.read_csv('/Users/mericgenc/Desktop/Python - Data Science/Miuul Data Scientist Path/04 - Recommendation Systems/00_datasets/movie_lens_dataset/movie.csv')
rating = pd.read_csv('/Users/mericgenc/Desktop/Python - Data Science/Miuul Data Scientist Path/04 - Recommendation Systems/00_datasets/movie_lens_dataset/rating.csv')

In [310]:
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [311]:
rating.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


### Adım 2:
rating veri setine Id’lere ait film isimlerini ve türünü movie veri setinden ekleyiniz.

In [312]:
df = rating.merge(movie, how='left', on='movieId')
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,2,3.5,2005-04-02 23:53:47,Jumanji (1995),Adventure|Children|Fantasy
1,1,29,3.5,2005-04-02 23:31:16,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi
2,1,32,3.5,2005-04-02 23:33:39,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
3,1,47,3.5,2005-04-02 23:32:07,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,3.5,2005-04-02 23:29:40,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


### Adım 3:
Toplam oy kullanılma sayısı 1000'in altında olan filmlerin isimlerini listede tutunuz ve veri setinden çıkartınız.

In [313]:
rating_counts = pd.DataFrame(df['title'].value_counts())
rare_movies = rating_counts[rating_counts['title']<1000].index
rare_movies

Index(['Bear, The (Ours, L') (1988)', 'One Night at McCool's (2001)',
       'Rosewood (1997)', 'Ted (2012)', 'Marked for Death (1990)',
       'Three to Tango (1999)', 'Adam's Rib (1949)',
       'My Super Ex-Girlfriend (2006)',
       'Italian for Beginners (Italiensk for begyndere) (2000)',
       'Husbands and Wives (1992)',
       ...
       'The Magnificent Trio (1966)', 'Five Weeks in a Balloon (1962)',
       'The Lost Squadron (1932)',
       'Under the Stars (Bajo las estrellas) (2007)',
       'Payment Deferred (1932)', 'Richard III (1912)',
       'The Nautical Chart (2007)', 'Two Arabian Knights (1927)',
       'Night Flight (1933)', 'Série noire (1979)'],
      dtype='object', length=23570)

In [314]:
common_movies = df[~df['title'].isin(rare_movies)]
common_movies.head()

,userId,movieId,rating,timestamp,title,genres
0,1,2,3.5,2005-04-02 23:53:47,Jumanji (1995),Adventure|Children|Fantasy
1,1,29,3.5,2005-04-02 23:31:16,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi
2,1,32,3.5,2005-04-02 23:33:39,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
3,1,47,3.5,2005-04-02 23:32:07,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,3.5,2005-04-02 23:29:40,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


### Adım 4:
index'te userID'lerin sutunlarda film isimlerinin ve değer olarak ratinglerin bulunduğu dataframe için pivot table oluşturunuz.

In [315]:
user_movie_df = common_movies.pivot_table(index='userId', columns='title', values='rating')
user_movie_df.head()

title,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),...,Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zulu (1964),[REC] (2007),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Adım 5:
Yapılan tüm işlemleri fonksiyonlaştırınız.

In [316]:
def create_user_movie_df():
    import pandas as pd
    movie = pd.read_csv('/Users/mericgenc/Desktop/Python - Data Science/Miuul Data Scientist Path/04 - Recommendation Systems/00_datasets/movie_lens_dataset/movie.csv')
    rating = pd.read_csv('/Users/mericgenc/Desktop/Python - Data Science/Miuul Data Scientist Path/04 - Recommendation Systems/00_datasets/movie_lens_dataset/rating.csv')
    df = rating.merge(movie, how='left', on='movieId')
    rating_counts = pd.DataFrame(df['title'].value_counts())
    rare_movies = rating_counts[rating_counts['title']<1000].index
    common_movies = df[~df['title'].isin(rare_movies)]
    user_movie_df = common_movies.pivot_table(index='userId', columns='title', values='rating')
    return user_movie_df
    

In [317]:
user_movie_df = create_user_movie_df()

In [318]:
user_movie_df.head()

title,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),...,Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zulu (1964),[REC] (2007),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [319]:
user_movie_df.tail()

title,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),...,Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zulu (1964),[REC] (2007),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
138489,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138491,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138492,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138493,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.5,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Görev 2: Öneri Yapılacak Kullanıcının İzlediği Filmlerin Belirlenmesi

### Adım 1:
Rastgele bir kullanıcı id’si seçiniz.

In [320]:
#Inception'ı izleyen bir kullanıcı seçiyorum
user_movie_df['Inception (2010)'].sort_values(ascending=False)

userId
69055     5.0
66049     5.0
66385     5.0
66343     5.0
66320     5.0
         ... 
138489    NaN
138490    NaN
138491    NaN
138492    NaN
138493    NaN
Name: Inception (2010), Length: 138493, dtype: float64

In [321]:
random_user = 69055

### Adım 2:
Seçilen kullanıcıya ait gözlem birimlerinden oluşan random_user_df adında yeni bir dataframe oluşturunuz.

In [322]:
random_user_df = user_movie_df[user_movie_df.index==random_user]

### Adım3:
Seçilen kullanıcıların oy kullandığı filmleri movies_watched adında bir listeye atayınız

In [323]:
random_user_df

title,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),...,Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zulu (1964),[REC] (2007),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
69055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [324]:
movies_watched = random_user_df.columns[random_user_df.notna().any()].to_list()

In [325]:
movies_watched

["Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)",
 'American Beauty (1999)',
 'Benny & Joon (1993)',
 'Beverly Hills Cop (1984)',
 'Blade Runner (1982)',
 'Casablanca (1942)',
 'Dracula: Dead and Loving It (1995)',
 'Eternal Sunshine of the Spotless Mind (2004)',
 'Fight Club (1999)',
 'Frida (2002)',
 'Harry Potter and the Deathly Hallows: Part 1 (2010)',
 'History of the World: Part I (1981)',
 'Inception (2010)',
 'Jewel of the Nile, The (1985)',
 "King's Speech, The (2010)",
 'Lord of the Rings: The Fellowship of the Ring, The (2001)',
 'Lord of the Rings: The Return of the King, The (2003)',
 'Lord of the Rings: The Two Towers, The (2002)',
 'Matrix, The (1999)',
 'Megamind (2010)',
 'Memento (2000)',
 'Mickey Blue Eyes (1999)',
 'Misérables, Les (1995)',
 'Moonstruck (1987)',
 'Mystic Pizza (1988)',
 'Runaway Bride (1999)',
 'Saturday Night Fever (1977)',
 "St. Elmo's Fire (1985)",
 'Star Wars: Episode IV - A New Hope (1977)',
 'Tangled (2010)',
 'WALL·E (2008)',
 "William

## Görev 3: Aynı Filmleri İzleyen Diğer Kullanıcıların Verisine ve Id'lerine Erişilmesi

### Adım 1:
Seçilen kullanıcının izlediği fimlere ait sutunları user_movie_df'ten seçiniz ve movies_watched_df adında yeni bir dataframe oluşturunuz.

In [326]:
movies_watched_df = user_movie_df[movies_watched]
movies_watched_df.head()

title,"Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)",American Beauty (1999),Benny & Joon (1993),Beverly Hills Cop (1984),Blade Runner (1982),Casablanca (1942),Dracula: Dead and Loving It (1995),Eternal Sunshine of the Spotless Mind (2004),Fight Club (1999),Frida (2002),...,"Misérables, Les (1995)",Moonstruck (1987),Mystic Pizza (1988),Runaway Bride (1999),Saturday Night Fever (1977),St. Elmo's Fire (1985),Star Wars: Episode IV - A New Hope (1977),Tangled (2010),WALL·E (2008),William Shakespeare's Romeo + Juliet (1996)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN
2,NaN,3.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN


### Adım 2:
Her bir kullancının seçili user'in izlediği filmlerin kaçını izlediğini bilgisini taşıyan user_movie_count adında yeni bir dataframe
oluşturunuz.

In [327]:
user_movie_count = pd.DataFrame(movies_watched_df.count(axis=1)).reset_index()
user_movie_count.columns = ["userId", "movie_count"]
user_movie_count

,userId,movie_count
0,1,7
1,2,3
2,3,4
3,4,0
4,5,1
...,...,...
138488,138489,5
138489,138490,2
138490,138491,0
138491,138492,6


### Adım3: 
Seçilen kullanıcının oy verdiği filmlerin yüzde 60 ve üstünü izleyenlerin kullanıcı id’lerinden users_same_movies adında bir liste oluşturunuz.

In [328]:
len(movies_watched)*60/100

19.2

In [329]:
users_same_movies = user_movie_count[user_movie_count['movie_count']>len(movies_watched)*60/100].index

In [330]:
users_same_movies

Int64Index([   207,    585,    613,    774,   1848,   2260,   2396,   2656,
              2668,   2914,
            ...
            136598, 136874, 136988, 137201, 137276, 137342, 137685, 137804,
            138161, 138207],
           dtype='int64', length=657)

## Görev 4: Öneri Yapılacak Kullanıcı ile En Benzer Kullanıcıların Belirlenmesi

### Adım 1: 
user_same_movies listesi içerisindeki seçili user ile benzerlik gösteren kullanıcıların id’lerinin bulunacağı şekilde movies_watched_df dataframe’ini filtreleyiniz.

In [331]:
final_df = pd.concat([movies_watched_df[movies_watched_df.index.isin(users_same_movies)],
                      random_user_df[movies_watched]])

### Adım 2:
Kullanıcıların birbirleri ile olan korelasyonlarının bulunacağı yeni bir corr_df dataframe’i oluşturunuz.

In [332]:
corr_df = final_df.T.corr().unstack().sort_values().drop_duplicates()
corr_df = pd.DataFrame(corr_df, columns=["corr"])
corr_df.index.names = ['user_id_1', 'user_id_2']
corr_df = corr_df.reset_index()
corr_df

,user_id_1,user_id_2,corr
0,64059,19962,-1.0
1,76629,39345,-1.0
2,23522,3317,-1.0
3,40846,68025,-1.0
4,34401,14704,-1.0
...,...,...,...
12056,120571,95546,1.0
12057,60761,80919,1.0
12058,125977,131433,1.0
12059,136267,52008,1.0


### Adım 3:
Seçili kullanıcı ile yüksek korelasyona sahip (0.65’in üzerinde olan) kullanıcıları filtreleyerek top_users adında yeni bir dataframe oluşturunuz.

In [333]:
corr_df[(corr_df['user_id_1']==random_user)]

,user_id_1,user_id_2,corr
21,69055,774,-0.994850
71,69055,21397,-0.973027
72,69055,67442,-0.970725
162,69055,115603,-0.939895
386,69055,8962,-0.862976
...,...,...,...
10915,69055,35513,0.711675
11013,69055,13665,0.733699
11354,69055,69849,0.804400
11754,69055,12792,0.906327


In [334]:
top_users = corr_df[(corr_df["user_id_1"] == random_user) & (corr_df["corr"] >= 0.65)][
    ["user_id_2", "corr"]].reset_index(drop=True)

In [335]:
top_users = top_users.sort_values(by='corr', ascending=False)

In [336]:
top_users.rename(columns={"user_id_2": "userId"}, inplace=True)

In [337]:
top_users

,userId,corr
5,31243,0.907045
4,12792,0.906327
3,69849,0.804400
2,13665,0.733699
1,35513,0.711675
0,68557,0.657848


### Adım 4:
top_users dataframe’ine rating veri seti ile merge ediniz.

In [338]:
rating.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [339]:
top_users_ratings = top_users.merge(rating[["userId", "movieId", "rating"]], how='inner')

In [340]:
top_users_ratings

,userId,corr,movieId,rating
0,31243,0.907045,50,4.0
1,31243,0.907045,260,0.5
2,31243,0.907045,356,3.0
3,31243,0.907045,367,3.5
4,31243,0.907045,784,0.5
...,...,...,...,...
997,68557,0.657848,82167,1.5
998,68557,0.657848,84772,4.0
999,68557,0.657848,86898,5.0
1000,68557,0.657848,89745,1.0


## Görev 5: Weighted Average Recommendation Score'un Hesaplanması ve İlk 5 Filmin Tutulması

### Adım 1:
Her bir kullanıcının corr ve rating değerlerinin çarpımından oluşan weighted_rating adında yeni bir değişken oluşturunuz.

In [341]:
top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['rating']
top_users_ratings.head()

,userId,corr,movieId,rating,weighted_rating
0,31243,0.907045,50,4.0,3.628178
1,31243,0.907045,260,0.5,0.453522
2,31243,0.907045,356,3.0,2.721134
3,31243,0.907045,367,3.5,3.174656
4,31243,0.907045,784,0.5,0.453522


### Adım 2:
Film id’si ve her bir filme ait tüm kullanıcıların weighted rating’lerinin ortalama değerini içeren recommendation_df adında yeni bir
dataframe oluşturunuz.

In [342]:
recommendation_df = top_users_ratings.groupby('movieId').agg({'weighted_rating':'mean'}).reset_index()

### Adım 3:
recommendation_df içerisinde weighted rating'i 3.5'ten büyük olan filmleri seçiniz ve weighted rating’e göre sıralayınız.

In [343]:
movies_to_be_recommend = recommendation_df[recommendation_df['weighted_rating']>3.5].sort_values('weighted_rating',ascending=False)

### Adım 4:
movie veri setinden film isimlerini getiriniz ve tavsiye edilecek ilk 5 filmi seçiniz.

In [344]:
movies_to_be_recommend.merge(movie[["movieId", "title"]], on='movieId').head(5)

,movieId,weighted_rating,title
0,2673,4.535223,Eternity and a Day (Mia aoniotita kai mia mera...
1,98017,4.535223,"Late Quartet, A (2012)"
2,88129,4.535223,Drive (2011)
3,4237,4.535223,"Gleaners & I, The (Les glaneurs et la glaneuse..."
4,81751,4.535223,Uncle Boonmee Who Can Recall His Past Lives (L...


# Item Based Recommendation

## Görev 1: Kullanıcının izlediği en son ve en yüksek puan verdiği filme göre item-based öneri yapınız.

### Adım 1:
movie, rating veri setlerini okutunuz.

In [345]:
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [346]:
rating.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


### Adım 2:
Seçili kullanıcının 5 puan verdiği filmlerden puanı en güncel olan filmin id'sinin alınız.

In [347]:
movie_id = rating[(rating['userId']==random_user) & (rating['rating']==5)].sort_values('timestamp', ascending=False)['movieId'][0:1].values[0]

### Adım 3:
User based recommendation bölümünde oluşturulan user_movie_df dataframe’ini seçilen film id’sine göre filtreleyiniz.

In [348]:
movie_name = movie[movie['movieId']==movie_id]['title'].values[0]

In [349]:
movie_name = user_movie_df[movie_name]

### Adım 4:
Filtrelenen dataframe’i kullanarak seçili filmle diğer filmlerin korelasyonunu bulunuz ve sıralayınız.

In [350]:
recommendation = user_movie_df.corrwith(movie_name).sort_values(ascending=False).head(10)
recommendation

title
Memento (2000)                                                 1.000000
Machinist, The (Maquinista, El) (2004)                         0.346076
Paradise Lost: The Child Murders at Robin Hood Hills (1996)    0.330017
Usual Suspects, The (1995)                                     0.318643
Run Lola Run (Lola rennt) (1998)                               0.311826
Fight Club (1999)                                              0.308780
Following (1998)                                               0.307218
Prestige, The (2006)                                           0.305858
Eternal Sunshine of the Spotless Mind (2004)                   0.303271
Inception (2010)                                               0.302969
dtype: float64

### Adım 5:
Seçili film’in kendisi haricinde ilk 5 film’I öneri olarak veriniz.

In [351]:
top_5 = recommendation.index[1:6].to_list()
top_5

['Machinist, The (Maquinista, El) (2004)',
 'Paradise Lost: The Child Murders at Robin Hood Hills (1996)',
 'Usual Suspects, The (1995)',
 'Run Lola Run (Lola rennt) (1998)',
 'Fight Club (1999)']